# Trip Data Aggregation
## Agrupar por colunas
1. year
2. mounth
3. pickup location ID
4. drop off location ID

## Agregar as colunas
1. total trip count
2. total fare amount

## Propósito do notebook

#### Demonstrar a integração entre Spark pool e o Serverless SQL pool
1. criar tabela agregada no Spark pool
2. acessar o dados via Serverless SQL pool 

In [1]:
#Define o caminho das pastas. 
bronze_folder_path = 'abfss://nyt-data@synapsenytdl.dfs.core.windows.net/raw'
silver_folder_path = 'abfss://nyt-data@synapsenytdl.dfs.core.windows.net/silver'
gold_folder_path = 'abfss://nyt-data@synapsenytdl.dfs.core.windows.net/gold'

StatementMeta(, , , SessionError, )

InvalidHttpRequestToLivy: Your Spark job requested 12 vcores. However, the workspace has a 0 core limit. Try reducing the numbers of vcores requested or increasing your vcore quota. HTTP status code: 400. Trace ID: 317993a8-dd3a-464a-b271-703cf676c279.

In [ ]:
#Define a configuração do spark para obter as colunas particionadas ano e mês como strings em vez de inteiros
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")

StatementMeta(, , , Cancelled, )

In [ ]:
%sql
-- Cria a base

CREATE DATABASE IF NOT EXISTS nyc_taxi_ldw_spark
LOCATION 'nyt-data@synapsenytdl.dfs.core.windows.net/gold';

StatementMeta(, , , Cancelled, )

In [ ]:
# Lê os dados "prata" a serem processados
trip_data_green_df = spark.read.parquet(f"{silver_folder_path}/trip_data_green") 

StatementMeta(, , , Cancelled, )

In [ ]:
# Executa as agregações necessárias
# 1. Conta o total de viagens
# 2. Mostra o valor total da tarifa
from pyspark.sql.functions import *

trip_data_green_agg_df = trip_data_green_df \
                        .groupBy("year", "month", "pu_location_id", "do_location_id") \
                        .agg(count(lit(1)).alias("total_trip_count"),
                        round(sum("fare_amount"), 2).alias("total_fare_amount"))

In [ ]:
# Grava os dados agregados na tabela "ouro" para consumo

trip_data_green_agg_df.write.mode("overwrite").partitionBy("year", "month").format("parquet").saveAsTable("nyt_ldw_spark.trip_data_green_agg")